# Tests for Husky PLL phase consistency.

Checks whether relative phase between target and ADC clocks is consistent: i.e. whether a given set of `scope.clock` parameters always produces the same phase.

Covers issues [490](https://github.com/newaetech/chipwhisperer/issues/490) and [499](https://github.com/newaetech/chipwhisperer/issues/499).

Does NOT check whether the phase is correct (e.g. [issue 501](https://github.com/newaetech/chipwhisperer/issues/501) -- that's covered in the part 2 notebook.

Should be run whenever changes are made to `ChipWhispererHuskyClocks.py`.

Could be a `pytest` script, however the ability to plot the clocks and visualize can be really useful when debugging.

In [ ]:
import chipwhisperer as cw

In [ ]:
scope = cw.scope()

In [ ]:
scope.default_setup()

# There are two cases to cover:
1. `EXTCLK = True`: target-supplied clock, from a CW305. Could be adapted to use a different target, but we need to be able to set its clock over a wide range.
2. `EXTCLLK = False`: no target needed; PLL uses 12 MHz XTAL reference.

In [ ]:
#EXTCLK = True
EXTCLK = False

In [ ]:
if EXTCLK:
    # setting PLL won't work without a bitfile?
    target = cw.target(scope, cw.targets.CW305, fpga_id='100t')
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)
    target.pll.pll_outfreq_set(10e6, 1)
    MAXCLOCK = 167e6
    #MAXCLOCK = 99e6 # NOTE: if allow_rdiv = False, can't go above 100M!
    scope.clock.clkgen_freq = 10e6
    scope.clock.clkgen_src = 'extclk'
else:
    MAXCLOCK = 200e6
    target = None
    scope.clock.clkgen_src = 'system'


OVERSAMP = 20
scope.LA.enabled = True
if EXTCLK:
    scope.LA.clk_source = 'target'
else:
    scope.LA.clk_source = 'pll'
scope.LA.clkgen_enabled = False
scope.LA.oversampling_factor = OVERSAMP
scope.LA.clkgen_enabled = True
scope.LA.capture_group = 'CW 20-pin'
scope.LA.capture_depth = 200 
assert scope.LA.locked

When `EXTCLK = True`, there would be a bunch of `scope_logger.errors()` due to clock/mul settings that can't be achieved; this silences them so that the output isn't cluttered.

For the same reason, we mute warnings.

**If debugging, unset `scope.clock._quiet` to see all errors, and turn warnings back on!**

In [ ]:
cw.scope_logger.setLevel(cw.logging.ERROR)
#cw.scope_logger.setLevel(cw.logging.WARNING)

if EXTCLK:
    scope.clock._quiet = True
else:
    scope.clock._quiet = False

In [ ]:
def setup(freq, mul, phase, allow_rdiv=False, update_mul=True):
    global all_settings
    if EXTCLK:
        target.pll.pll_outfreq_set(freq, 1)
    # randomly order the mul/freq updates:
    if random.randint(0,1):
        if update_mul: scope.clock.adc_mul = mul
        scope.clock.clkgen_freq = freq
    else:
        scope.clock.clkgen_freq = freq
        if update_mul: scope.clock.adc_mul = mul
    if phase is not None:
        #print('applying provided phase: %f' % phase)
        scope.clock.adc_phase = phase
    else:
        maxphase = scope.clock.pll.max_phase_percent
        phase = random.random()*maxphase
        if random.randint(0,1) and not EXTCLK:
            phase = -phase
        #print('applying chosen phase: %f' % phase)
        scope.clock.adc_phase = phase
    assert scope.clock.pll.pll_locked
    setting = {}
    setting['freq'] = freq
    setting['mul'] = mul
    setting['phase'] = phase
    setting['allow_rdiv'] = allow_rdiv
    setting['update_mul'] = update_mul
    setting['pll_settings'] = str(scope.clock.pll)
    setting['params'] = list(scope.clock.pll.parameters)
    all_settings.append(setting)
    return scope.clock.adc_phase

def set_oversamp(oversamp):
    scope.LA.clkgen_enabled = False
    scope.LA.oversampling_factor = oversamp
    scope.LA.clkgen_enabled = True
    assert scope.LA.locked

def get_fail_package(ID, FREQ, OVERSAMP, TOL, MUL, INPUT_PHASE, NEW_PHASE, NEWFREQ, NEWMUL, ref_phase, ref_adc, ref_ref, new_phase, new_adc, new_ref, ref_settings, new_settings, prev_settings, ref_sampling_clock, new_sampling_clock):
    packed = {}
    packed['ID'] = ID
    packed['FREQ'] = FREQ
    packed['OVERSAMP'] = OVERSAMP
    packed['TOL'] = TOL
    packed['MUL'] = MUL
    packed['INPUT_PHASE'] = INPUT_PHASE
    packed['NEW_PHASE'] = NEW_PHASE
    packed['NEWFREQ'] = NEWFREQ
    packed['NEWMUL'] = NEWMUL
    packed['ref_phase'] = ref_phase
    packed['new_phase'] = new_phase
    packed['ref_settings'] = ref_settings
    packed['new_settings'] = new_settings
    packed['prev_settings'] = prev_settings
    packed['ref_sampling_clock'] = ref_sampling_clock
    packed['new_sampling_clock'] = new_sampling_clock

    offset = find0to1trans(ref_ref)[0]
    packed['ref_adc'] = ref_adc[offset:]
    packed['ref_ref'] = ref_ref[offset:]

    offset = find0to1trans(new_ref)[0]
    packed['new_adc'] = new_adc[offset:]
    packed['new_ref'] = new_ref[offset:]
    return packed

def find0to1trans(data):
    pattern = [0,1]
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

def get_clocks(extclk=False):
    done = False
    count = 0
    while not done and count < 30:
        scope.LA.arm()
        scope.LA.trigger_now()
        raw = scope.LA.read_capture_data()
        adcclock = scope.LA.extract(raw, 8)
        if extclk:
            refclock = scope.LA.extract(raw, 4)
        else:
            refclock = scope.LA.extract(raw, 5)
        
        edges = find0to1trans(refclock)
        if len(edges) > 1:
            ref_edge = edges[1]
        else:
            ref_edge = edges[0]    
        
        try:
            #adc_ref_delta = find0to1trans(adcclock[ref_edge:])[0]
            adc_edges = find0to1trans(adcclock)
            adc_ref_delta = abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge)
            done = True
        except:
            # not sure why but sometimes the ADC clock comes back all zeros; could be an issue with the PLL or with the LA?
            # what's very strange is that this doesn't happen often, but when it does, adcclock is always all zeros, and 
            # the capture is re-attempted exactly 19 times before it's successful!
            if all(c == 0 for c in adcclock):
                adcclock = 'all zeros'
            print('could not find delta; ref_edge=%3d, lock status=%s; adcclock=%s; trying again' % (ref_edge, scope.clock.pll.pll_locked, adcclock))
            assert scope.LA.locked
            assert scope.clock.pll.pll_locked
            time.sleep(0.5)
            count += 1
    return adc_ref_delta, adcclock, refclock

# Main Test Loop

Note that here we do NOT concern ourselves at all with whether the phase is **correct**; all we care is whether the phase is **consistent**.

The approach is:
1. Pick a random (but valid) frequency and `adc_mul`; measure the clocks' relative phase.
2. Change the clock frequency and `adc_mul` to other values, then return to the original setting from step 1 and check if the phase is the same that it was before.

The `REPS` parameter detemines how long the test is. The loop runs at around 20 seconds per rep.

In [ ]:
#REPS = 10
REPS = 2000

In [ ]:
EXTCLK

In [ ]:
scope.trace.clock._warning_frequency = 303e6

In [ ]:
from tqdm.notebook import tnrange, tqdm
import numpy as np
import time
import random

TEST_CLOCK_CHANGE = True
TEST_MUL_CHANGE = True

la_skipped = 0
setup_skipped = 0
new_phase_skipped = 0
phase_problems = 0
change_problems_f = 0 
change_problems_m = 0
fails = 0
passes = 0
saved_fails = []
all_settings = []
freqmuls = []

pbar = tqdm(total=2*REPS*3, desc='Passing')
fbar = tqdm(total=2*REPS*3, desc='Failing')
sbar = tqdm(total=REPS, desc='Skipped')

for i in tnrange(REPS):
    prev_settings = str(scope.clock.pll)
    FREQ = random.randint(5e2, 20e2)*1e4 # lower resolution gets much faster setting of the CW305 PLL
    OVERSAMP = int(300e6//FREQ)
    if EXTCLK:
        # in this case, the ADC clock and target-generated clock can both be shifting around relative to each other, so we need to tolerate a larger deviance:
        mintol = 2
    else:
        mintol = 1
    TOL = max(mintol, OVERSAMP//30)
    maxmul = int(np.floor(25e6/FREQ))
    MUL = random.randint(1, maxmul)
    freqmuls.append([FREQ, MUL])
    PHASE = None # setup() will pickup a random valid phase
    try:
        INPUT_PHASE = setup(FREQ, MUL, PHASE)
        set_oversamp(OVERSAMP)
        time.sleep(0.5)
        if abs(scope.LA.sampling_clock_frequency/FREQ - OVERSAMP) / OVERSAMP * 100 > 1:
            la_skipped += 1
            sbar.update(1)
            continue
    except Exception as e:
        print('failed to setup: %s' % e)
        setup_skipped += 1
        continue
    ref_phase, ref_adc, ref_ref = get_clocks(EXTCLK)
    ref_settings = str(scope.clock.pll)
    ref_params = scope.clock.pll.parameters
    ref_sampling_clock = scope.LA.sampling_clock_frequency

    if ref_phase is None:
        print('***** Could not get ref_phase')
        continue
    for test in ['test_clock_change', 'test_mul_change']:
        for j in range(3):
            try:
                if test == 'test_clock_change':
                    #newfreq = random.uniform(5e6, MAXCLOCK/scope.clock.adc_mul)
                    newfreq = random.randint(5e2, MAXCLOCK/scope.clock.adc_mul//1e4)*1e4 # lower resolution gets much faster setting of the CW305 PLL
                    newmul = MUL
                else:
                    newfreq = FREQ
                    maxmul = int(np.floor(200e6/FREQ))
                    newmul = random.randint(1, maxmul)
                freqmuls.append([newfreq, newmul])
                scope.LA.clkgen_enabled = False
                setup(newfreq, newmul, 0)
                try:
                    setup(FREQ, MUL, INPUT_PHASE)
                    set_oversamp(OVERSAMP)
                except Exception as e:
                    print('could not re-apply phase in %s: %s' % (test, e))
                    phase_problems += 1
                    continue
                new_settings = str(scope.clock.pll)
                new_params = scope.clock.pll.parameters
                if new_params != ref_params:
                    print('changed settings (%s); all_setting size: %d' % (test, len(all_settings)))
                    if test == 'test_clock_change':
                        change_problems_f += 1
                    else:
                        change_problems_m += 1
                new_phase, new_adc, new_ref = get_clocks(EXTCLK)
                new_sampling_clock = scope.LA.sampling_clock_frequency
                if new_phase is None:
                    new_phase_skipped += 1
                    sbar.update(1)
                    continue
                if abs(new_phase - ref_phase) > TOL:
                    print('ID %d: FREQ=%12d, OVERSAMP=%3d, TOL=%2d, MUL=%2d, INPUT_PHASE=%f' % (fails, FREQ, OVERSAMP, TOL, MUL, INPUT_PHASE), end='')
                    if test == 'test_clock_change':
                        print('***** Got unexpected phase %d (reference phase = %d) when changing clock frequency (%f) on iteration %d' % (new_phase, ref_phase, newfreq, j))
                    else:
                        print('***** Got unexpected phase %d (reference phase = %d) when changing adc_mul (%d) on iteration %d' % (new_phase, ref_phase, newmul, j))
                    saved_fails.append(get_fail_package(fails, FREQ, OVERSAMP, TOL, MUL, INPUT_PHASE, scope.clock.adc_phase, newfreq, MUL, ref_phase, ref_adc, ref_ref, new_phase, new_adc, new_ref, ref_settings, new_settings, prev_settings, ref_sampling_clock, new_sampling_clock))
                    fails += 1
                    fbar.update(1)
                else:
                    passes += 1
                    pbar.update(1)
            except Exception as e:
                new_phase_skipped += 1
                #print('new_phase_skipped: %s' % e)
                scope.clock.pll._registers_cached = False
                sbar.update(1)

Note that it's normal for the "passing" bar to not finish at 100%, due to "skips".

Check that there are no failures and that we don't have "too many" skips:

In [ ]:
assert fails == 0
assert passes / (la_skipped + setup_skipped + new_phase_skipped) > 20

## Check for issue #499:

In [ ]:
if EXTCLK:
    input_dividers = 0
    for i,s in enumerate(all_settings):
        p = s['params']
        muldiv = 1/p[0]*p[1]*p[2]/p[3]/p[4]
        if muldiv != 1:
            print('looks like issue #499: %d %f' % (i,muldiv))
        
        if p[0] != 1 and s['freq'] <= 100e6:
            print('oops! %d has input divider != 1!' % i)
            input_dividers += 1
    assert input_dividers == 0

**This concludes the test; the rest of this notebook is to help diagnose failures.**

# Investigate issues found by main test loop

## 1. plot what we got

In [ ]:
len(saved_fails)

Pick a failure:

In [ ]:
fail_id = 1

In [ ]:
saved_fails[fail_id]['ref_settings'] == saved_fails[fail_id]['new_settings']

In [ ]:
print(saved_fails[fail_id]['prev_settings'])
print(saved_fails[fail_id]['ref_settings'])
print(saved_fails[fail_id]['new_settings'])

In [ ]:
saved_fails[fail_id]['ref_settings'] == saved_fails[fail_id]['prev_settings']

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
import numpy as np

output_notebook(INLINE)
#output_notebook()

In [ ]:
samples = min(len(saved_fails[fail_id]['ref_adc']), len(saved_fails[fail_id]['new_adc']))
xrange = list(range(samples))

p = figure(width=1800)

p.line(xrange, saved_fails[fail_id]['ref_ref'][:samples], line_color='blue')
p.line(xrange, saved_fails[fail_id]['ref_adc'][:samples] - 2, line_color='red')

p.line(xrange, saved_fails[fail_id]['new_ref'][:samples] - 6, line_color='green')
p.line(xrange, saved_fails[fail_id]['new_adc'][:samples] - 4, line_color='brown')

p.line(xrange, np.asarray(saved_fails[fail_id]['ref_adc'][:samples]) - np.asarray(saved_fails[fail_id]['new_adc'][:samples])- 8, line_color='black', line_width=2)

show(p)

In [ ]:
edges = find0to1trans(saved_fails[fail_id]['ref_ref'])
ref_edge = edges[1]
adc_edges = find0to1trans(saved_fails[fail_id]['ref_adc'])
print(abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge))

edges = find0to1trans(saved_fails[fail_id]['new_ref'])
ref_edge = edges[1]
adc_edges = find0to1trans(saved_fails[fail_id]['new_adc'])
print(abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge))


In [ ]:
edges = find0to1trans(saved_fails[fail_id]['ref_ref'])
ref_edge = edges[1]
adc_edges = find0to1trans(saved_fails[fail_id]['ref_adc'])
print(abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge))

In [ ]:
ref_edge, adc_edges

In [ ]:
edges = find0to1trans(saved_fails[fail_id]['new_ref'])
ref_edge = edges[1]
adc_edges = find0to1trans(saved_fails[fail_id]['new_adc'])
print(abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge))

In [ ]:
ref_edge, adc_edges

In [ ]:
phase

In [ ]:
saved_fails[0]

## 2. repeat with same parameters

In [ ]:
INPUT_PHASE

In [ ]:
fail_id = 0

In [ ]:
FREQ = saved_fails[fail_id]['FREQ']
OVERSAMP = saved_fails[fail_id]['OVERSAMP']
MUL = saved_fails[fail_id]['MUL']
INPUT_PHASE = saved_fails[fail_id]['INPUT_PHASE']

In [ ]:
saved_fails[fail_id]['INPUT_PHASE'], INPUT_PHASE

In [ ]:
saved_fails[fail_id]['ref_settings']

In [ ]:
setup(FREQ, MUL, INPUT_PHASE)
set_oversamp(OVERSAMP)

In [ ]:
phase, adc, ref = get_clocks(EXTCLK)
print(phase, saved_fails[fail_id]['ref_phase'])

In [ ]:
offset = find0to1trans(ref)[0]

In [ ]:
adc0 = adc[offset:]
ref0 = ref[offset:]

In [ ]:
scope.clock.adc_phase

In [ ]:
scope.clock.pll

In [ ]:
NEWFREQ = saved_fails[fail_id]['NEWFREQ']
NEWMUL = saved_fails[fail_id]['NEWMUL']

In [ ]:
setup(NEWFREQ, NEWMUL, 0)
setup(FREQ, MUL, INPUT_PHASE)

In [ ]:
pphase, padc, pref = get_clocks(EXTCLK)
print(pphase)

In [ ]:
scope.clock.pll

In [ ]:
saved_fails[fail_id]['new_settings']

In [ ]:
saved_fails[fail_id]['ref_settings']

In [ ]:
saved_fails[fail_id]['prev_settings']

In [ ]:
scope.clock.adc_phase

In [ ]:
offset = find0to1trans(pref)[0]

In [ ]:
adc2 = padc[offset:]
ref2 = pref[offset:]

In [ ]:
samples = min(len(adc0), len(adc2))
xrange = list(range(samples))

p = figure(width=1800)

p.line(xrange, ref0[:samples], line_color='blue')
p.line(xrange, adc0[:samples] - 2, line_color='red')

p.line(xrange, ref2[:samples] - 6, line_color='green')
p.line(xrange, adc2[:samples] - 4, line_color='brown')

p.line(xrange, np.asarray(adc0[:samples]) - np.asarray(adc2[:samples])- 8, line_color='black', line_width=2)

show(p)

# Don't check phase: just find cases for issue #499:

Runs faster than main test loop.

In [ ]:
REPS = 20000
all_settings = []
for i in tnrange(REPS):
    FREQ = random.randint(5e2, 20e2)*1e4 # lower resolution gets much faster setting of the CW305 PLL
    #OVERSAMP = int(300e6//FREQ)
    maxmul = int(np.floor(60e6/FREQ))
    MUL = random.randint(1, maxmul)
    freqmuls.append([FREQ, MUL])
    try:
        INPUT_PHASE = setup(FREQ, MUL, 0)    
    except Exception as e:
        print('failed to setup: %s' % e)
        setup_skipped += 1
        continue
    in_div, pll_mul, fb_prescale, prescale, out_div1, out_div3 = scope.clock.pll.parameters
    ratio = 1 / in_div * pll_mul * fb_prescale / prescale / out_div1
    if ratio != 1:
        print('*** Got one! iteration %d' % i)
    #else:
    #    print('%d ratio: %f' % (i, ratio))